In [1]:
# Install required packages
!pip install transformers datasets nltk rouge-score sacrebleu sentence-transformers sentencepiece fsspec==2025.3.2 bert-score --quiet

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.4 MB/s eta 0:00:00

In [2]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [3]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [5]:
!pip install -U gdown

# Use the folder ID from:
# https://drive.google.com/drive/folders/1QdxrYnelt9poi45eLT5xgObihDRb_OtV
!gdown --folder 1QdxrYnelt9poi45eLT5xgObihDRb_OtV -O /content/103080


Retrieving folder contents
Retrieving folder 1rzaJxn-bvoSXzs4Zhyac-YWjH-HxFUEM 1_Pooling
Processing file 1GwEAz43APaFICfzVB_VRreAEvfr1Ra20 config.json
Processing file 1p7CoqRUhOIqXYcTuY_d-qYvZLTjcpxki config_sentence_transformers.json
Processing file 1oUflzLJYyHewOzW_X_iDjsLxd8ppvYzx config.json
Processing file 1IcFz-ITRtZkY6AC-x9ieb5CAdnNrzmsT modules.json
Processing file 115xWQSGruGaZie1V8KVg4_-Xw2DjF8ah pytorch_model.bin
Processing file 1Q7s6JhhsWmoWP09_n6zR9jJM4fU0lTz- README.md
Processing file 1rn6Sue3wcp44qm3whgJtrVfBDkEd9BWH sentence_bert_config.json
Processing file 1BudqNXrBJMzy7tYPxVUezizSpAQs2TMi special_tokens_map.json
Processing file 1nWX9rbnv1V18rJN2tKrWNCI_Ulpp2-ds tokenizer_config.json
Processing file 1Cev_MvvPrZvQ87Z0WcjH96kTVgXY9xkc tokenizer.json
Processing file 1bkHtwyjLYLLwSHaSzvstD_dQWIKchF0B vocab.txt
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1GwEAz43

In [6]:
# ─── IMPORTS ──────────────────────────────────────────────────────────────
import os
import json
import hashlib
import numpy as np
import torch
import nltk

from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize
import sacrebleu
from sentence_transformers import util
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModel
from bert_score import score as bert_score

In [7]:
# Clone your repo and update base_dir
!git clone https://github.com/DrishtiShrrrma/codeclarity.git



Cloning into 'codeclarity'...
remote: Enumerating objects: 534, done.
remote: Counting objects: 100% (116/116), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 534 (delta 85), reused 24 (delta 24), pack-reused 418 (from 2)
Receiving objects: 100% (534/534), 8.91 MiB | 19.49 MiB/s, done.
Resolving deltas: 100% (296/296), done.


In [8]:
# ─── CONFIGURATION ─────────────────────────────────────────────────────────

backtranslation_dir = "backtranslations_cache"
os.makedirs(backtranslation_dir, exist_ok=True)

# Mapping for summary field name → Display name (used in logs, printouts)
json_field_to_lang = {
    "chinese":     "Chinese",
    "french":      "French",
    "spanish":     "Spanish",
    "portuguese":  "Portuguese",
    "arabic":      "Arabic",
    "hindi":       "Hindi"
}

# Mapping for Display name → M2M-100 language code (used for backtranslation)
m2m_lang_map = {
    "Chinese":     "zh",
    "French":      "fr",
    "Spanish":     "es",
    "Portuguese":  "pt",
    "Arabic":      "ar",
    "Hindi":       "hi"
}


In [9]:
# Load M2M-100 model
m2m_model_name = "facebook/m2m100_418M"
tok_m2m = AutoTokenizer.from_pretrained(m2m_model_name)
model_m2m = AutoModelForSeq2SeqLM.from_pretrained(m2m_model_name)
device = "cuda" if torch.cuda.is_available() else "cpu"
model_m2m = model_m2m.to(device)

# Caches
embedding_model     = None
bertscore_model     = None
bertscore_tokenizer = None
side_tokenizer      = None
side_model          = None

# ─── BACK-TRANSLATION FUNCTION ─────────────────────────────────────────────
def backtranslate_with_m2m(text, src_name):
    key = f"{src_name}_{hashlib.md5(text.encode()).hexdigest()}"
    cache_file = os.path.join(backtranslation_dir, key + ".txt")
    if os.path.exists(cache_file):
        return open(cache_file, 'r', encoding='utf-8').read()

    src_lang = m2m_lang_map.get(src_name)
    tgt_lang = "en"  # Always translating into English

    if src_lang is None:
        return text

    tok_m2m.src_lang = src_lang
    inputs = tok_m2m(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    generated_tokens = model_m2m.generate(**inputs, forced_bos_token_id=tok_m2m.get_lang_id(tgt_lang))
    output_text = tok_m2m.decode(generated_tokens[0], skip_special_tokens=True)

    with open(cache_file, 'w', encoding='utf-8') as f:
        f.write(output_text)

    return output_text

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

In [10]:
# ─── METRIC FUNCTIONS ──────────────────────────────────────────────────────
def compute_bertscore(refs, hyps):
    P, R, F1 = bert_score(
        hyps,
        refs,
        model_type="xlm-roberta-large",
        lang="en",
        rescale_with_baseline=False
        \
    )
    return {
        "precision": round(P.mean().item(), 4),
        "recall":    round(R.mean().item(), 4),
        "f1":        round(F1.mean().item(), 4)
    }

In [11]:
# ─── MEAN POOLING (for SIDE) ───────────────────────────────────────────────
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    mask = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * mask, 1) / torch.clamp(mask.sum(1), min=1e-9)


def compute_side_score(codes, hyps):
    global side_tokenizer, side_model
    if side_model is None:
        checkpoint = "/content/103080"  # <-- Change if needed
        side_tokenizer = AutoTokenizer.from_pretrained(checkpoint)
        side_model     = AutoModel.from_pretrained(checkpoint)
        if torch.cuda.is_available(): side_model = side_model.cuda()
        side_model.eval()
    scores = []
    for code, summ in zip(codes, hyps):
        enc = side_tokenizer([code, summ], padding=True, truncation=True, return_tensors="pt")
        if torch.cuda.is_available(): enc = {k:v.cuda() for k,v in enc.items()}
        with torch.no_grad(): out = side_model(**enc)
        pooled = mean_pooling(out, enc['attention_mask'])
        normed = torch.nn.functional.normalize(pooled, p=2, dim=1)
        scores.append(util.pytorch_cos_sim(normed[0], normed[1]).item())
    return round(float(np.mean(scores)),4)

In [12]:
def compute_meteor_score(refs, hyps):
    sc = []
    for r,h in zip(refs, hyps):
        rt = word_tokenize(r.lower()); ht = word_tokenize(h.lower())
        sc.append(meteor_score([rt], ht))
    return round(float(np.mean(sc)),4)

In [13]:
def compute_chrf_score(refs, hyps):
  refs = [r.lower() for r in refs]
  hyps = [h.lower() for h in hyps]

  res = sacrebleu.corpus_chrf(hyps, [refs], word_order=2)
  return round(res.score / 100, 4)


In [14]:
def compute_bleu_sacre(refs, hyps, lang_name):
    # Optionally choose a tokenizer based on the language
    tokenizer = "zh" if lang_name.lower() == "chinese" else "13a"

    score = sacrebleu.corpus_bleu(hyps, [refs], tokenize=tokenizer)
    return round(score.score / 100, 4)  # Normalize to 0–1 like nltk


In [15]:
def compute_all_metrics(refs, hyps, lang_name, code_lang):
    print(f"  Computing backtranslation-based metrics for {lang_name}...")
    bt = [backtranslate_with_m2m(h, lang_name) for h in hyps]
    smoothie = SmoothingFunction().method4
    rlist = [[r.split()] for r in refs]
    hlist = [h.split() for h in bt]
    bleu_nltk = corpus_bleu(rlist, hlist, smoothing_function=smoothie)
    bleu_sacre = compute_bleu_sacre(refs, bt, lang_name)
    bleu_diff = round(abs(bleu_nltk - bleu_sacre), 4)

    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    rl = [scorer.score(r, b)['rougeL'].fmeasure for r, b in zip(refs, bt)]

    return {
        "bleu4_nltk": round(bleu_nltk, 4),
        "bleu4_sacrebleu": bleu_sacre,
        "bleu4_diff": bleu_diff,
        "rougeL": round(np.mean(rl), 4),
        "meteor": compute_meteor_score(refs, bt),
        "chrf++": compute_chrf_score(refs, bt),
        "side_bt": compute_side_score(refs, bt)
    }


In [16]:
import os
import json
import pandas as pd

def run_evaluation():
    base_dir = "/content/codeclarity/generated_code_summaries/final-run-v2/codegemma-7b-it"
    model_folder = "codegemma-7b-it"
    model_path = base_dir
    all_results = []

    print(f"\n📦 Evaluating model: {model_folder}")

    for fname in os.listdir(model_path):
        if not fname.endswith('.json'):
            continue
        if "all_languages_combined" in fname:
            continue  # Skip combined file if exists

        summary_path = os.path.join(model_path, fname)
        code_lang = fname.split('_')[0]
        print(f"\n📄 File: {fname} | Programming language: {code_lang}")

        with open(summary_path, encoding='utf-8') as f:
            data = json.load(f)

        refs = [d['summary_english'] for d in data]

        for field, lang_name in json_field_to_lang.items():
            if f'summary_{field}' not in data[0]:
                print(f"  ⚠️ Field summary_{field} not found in dataset. Skipping {lang_name}.")
                continue

            hyps = [d.get(f'summary_{field}', '') for d in data]
            if not any(hyps):
                print(f"  ⛔ All summaries for {lang_name} are empty or missing. Skipping.")
                continue

            print(f"\n🔍 Evaluating {lang_name} summaries...")
            print("  ➤ Computing BERTScore...")
            bert = compute_bertscore(refs, hyps)
            print("  ➤ Computing direct multilingual embedding similarity (SIDE)...")
            side_original = compute_side_score(refs, hyps)
            metrics = compute_all_metrics(refs, hyps, lang_name, code_lang)
            side_drop = round(side_original - metrics["side_bt"], 4)

            print(f"\n📊 Results for {lang_name} → English:")
            print(f"    BERTScore (F1): {bert['f1']}")
            print(f"    SIDE Original (multilingual): {side_original}")
            print(f"    SIDE Backtranslated (English): {metrics['side_bt']}")
            print(f"    BLEU-4 NLTK (BT): {metrics['bleu4_nltk']}")
            print(f"    BLEU-4 SacreBLEU (BT): {metrics['bleu4_sacrebleu']}")
            print(f"    BLEU Difference: {metrics['bleu4_diff']}")
            print(f"    ROUGE-L (BT): {metrics['rougeL']}")
            print(f"    METEOR (BT): {metrics['meteor']}")
            print(f"    CHRF++ (BT): {metrics['chrf++']}")
            print("-" * 60)

            for i, entry in enumerate(data):
                sample_id = entry.get("id", f"{code_lang}_{i}")
                generated_summary = entry.get(f'summary_{field}', '')
                backtranslated_summary = backtranslate_with_m2m(generated_summary, lang_name)

                result = {
                    "sample_id": sample_id,
                    "model_folder_name": model_folder,
                    "model_name": entry.get("model_name", model_folder),
                    "programming_language": code_lang,
                    "language": lang_name,
                    "reference_summary": entry.get("summary_english", ""),
                    "generated_summary": generated_summary,
                    "backtranslated_summary": backtranslated_summary,
                    "bertscore_f1": bert["f1"],
                    "bertscore_precision": bert["precision"],
                    "bertscore_recall": bert["recall"],
                    "side_original": side_original,
                    "side_bt": metrics["side_bt"],
                    "side_drop": side_drop,
                    "bleu4_nltk": metrics["bleu4_nltk"],
                    "bleu4_sacrebleu": metrics["bleu4_sacrebleu"],
                    "bleu4_diff": metrics["bleu4_diff"],
                    "rougeL": metrics["rougeL"],
                    "meteor": metrics["meteor"],
                    "chrf++": metrics["chrf++"]
                }

                all_results.append(result)

    # ─── SAVE RESULTS ──────────────────────────────────────────────────────
    os.makedirs(backtranslation_dir, exist_ok=True)
    json_out = os.path.join(backtranslation_dir, 'all_scores.json')
    csv_out = os.path.join(backtranslation_dir, 'all_scores.csv')

    with open(json_out, 'w', encoding='utf-8') as f:
        json.dump(all_results, f, indent=2)

    df = pd.DataFrame(all_results)
    df.to_csv(csv_out, index=False)

    print(f"\n✅ All scores saved to:\n- JSON: {json_out}\n- CSV: {csv_out}")


In [17]:
if __name__ == "__main__":
    run_evaluation()



📦 Evaluating model: codegemma-7b-it

📄 File: ruby_summary_all_languages_codegemma-7b-it.json | Programming language: ruby

🔍 Evaluating Chinese summaries...
  ➤ Computing BERTScore...


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

  ➤ Computing direct multilingual embedding similarity (SIDE)...
  Computing backtranslation-based metrics for Chinese...

📊 Results for Chinese → English:
    BERTScore (F1): 0.8737
    SIDE Original (multilingual): 0.7847
    SIDE Backtranslated (English): 0.8312
    BLEU-4 NLTK (BT): 0.0259
    BLEU-4 SacreBLEU (BT): 0.1062
    BLEU Difference: 0.0803
    ROUGE-L (BT): 0.2853
    METEOR (BT): 0.2794
    CHRF++ (BT): 0.2572
------------------------------------------------------------

🔍 Evaluating French summaries...
  ➤ Computing BERTScore...
  ➤ Computing direct multilingual embedding similarity (SIDE)...
  Computing backtranslation-based metrics for French...

📊 Results for French → English:
    BERTScore (F1): 0.8728
    SIDE Original (multilingual): 0.8406
    SIDE Backtranslated (English): 0.9062
    BLEU-4 NLTK (BT): 0.0337
    BLEU-4 SacreBLEU (BT): 0.0719
    BLEU Difference: 0.0382
    ROUGE-L (BT): 0.3152
    METEOR (BT): 0.2181
    CHRF++ (BT): 0.276
---------------------

In [19]:
from google.colab import files

# Download JSON
files.download('backtranslations_cache/all_scores.json')

# Download CSV
files.download('backtranslations_cache/all_scores.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>